# 3.协程篇

多进程和多线程切换之间也是有资源浪费的，相比而言协程更轻量级

## 3.1.知识回顾

### 1.装饰器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9333792.html#2.Python装饰器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9333792.html#2.Python装饰器</a>

基础拓展篇已经讲的很透彻了，就不再雷同了，贴一个简单案例，然后扩展说说**`可迭代`、`迭代器`和`生成器`**

In [1]:
% time

from functools import wraps

def log(func):
    @wraps(func)
    def wrapper(*args,**kv):
        print("%s log_info..." % func.__name__)
        return func(*args,**kv)
    return wrapper

@log
def login_out():
    print("已经退出登录")

def main():
    # @wraps(func) 可以使得装饰前后，方法签名一致
    print(f"方法签名：{login_out.__name__}")
    login_out()
    
    # @wraps能让你通过属性 __wrapped__ 直接访问被包装函数
    login_out.__wrapped__() # 执行原来的函数

if __name__ == '__main__':
    main()

Wall time: 0 ns
方法签名：login_out
login_out log_info...
已经退出登录
已经退出登录


### 2.迭代器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#6.Python迭代器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9278573.html#6.Python迭代器</a>

过于基础的就不说了，简单说下，然后举一个`OOP`的`Demo`：
1. **判断是否可迭代：（能不能for遍历）**
    - `from collections.abc import Iterable`
    - `isinstance(xxx, Iterable)`
2. **判断是否是迭代器：（能不能`next(xxx)`遍历）**
    - `from collections.abc import Iterator`
    - `isinstance(xxx, Iterable)`
    - PS：迭代器是一定可以迭代的
3. **可迭代对象转迭代器：（生成器都是迭代器）**
    - 把`list、dict、str`等`Iterable`变成`Iterator`可以使用`iter()`函数 eg：**`iter([])`**（节省资源）
    - PS：生成器都是`Iterator`对象，但list、dict、str虽然是`Iterable`，却不是`Iterator`

**提醒一下：`from collections import Iterable, Iterator # 现在已经不推荐使用了（3.8会弃用）`**

查看一下`typing.py`的源码就知道了:
```PY
# 模仿collections.abc中的那些（Python3.7目前只是过渡的兼容版，没有具体实现）
def _alias(origin, params, inst=True):
    return _GenericAlias(origin, params, special=True, inst=inst)

T_co = TypeVar('T_co', covariant=True)  # Any type covariant containers.

Iterable = _alias(collections.abc.Iterable, T_co)
Iterator = _alias(collections.abc.Iterator, T_co)
```

之前说了个 <a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#7.1.IEnumerator-%E5%92%8C-IEnumerable" target="_blank">CSharp 的 OOP Demo</a>，这次来个`Python`的，我们来一步步演变：

In [2]:
% time

# 导入相关模块
from collections.abc import Iterable, Iterator
# from collections import Iterable, Iterator # 现在已经不推荐使用了（3.8会弃用）

Wall time: 0 ns


In [3]:
# 定义一个Class
class MyArray(object):
    pass

In [4]:
# 是否可迭代 False
isinstance(MyArray(),Iterable)

False

In [5]:
# 是否是迭代器 False
isinstance(MyArray(),Iterator)

False

In [6]:
# 如果Class里面含有`__iter__`方法就是可迭代的

In [7]:
# 重新定义测试：
class MyArray(object):
    def __iter__(self):
        pass

# 是否可迭代 False
isinstance(MyArray(),Iterable)

True

In [8]:
# 是否是迭代器 False
isinstance(MyArray(),Iterator)

False

**这时候依然不是迭代器**

这个可以类比C#：
1. 能不能foreach就看你遍历对象有没有实现IEnumerable，就说明你是不是一个可枚举类型（enumerator type）
2. 是不是个枚举器（enumerator）就看你实现了IEnumerator接口没

```csharp
// 能不能foreach就看你遍历对象有没有实现IEnumerable，就说明你是不是一个可枚举类型
public interface IEnumerable
{
    IEnumerator GetEnumerator();
}

// 是不是个枚举器（enumerator）就看你实现了IEnumerator接口没
public interface IEnumerator
{
    object Current { get; }

    bool MoveNext();

    void Reset();
}
```
先看看Python对于的类吧：
```py
# https://github.com/lotapp/cpython3/blob/master/Lib/_collections_abc.py
class Iterable(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __iter__(self):
        while False:
            yield None

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterable:
            return _check_methods(C, "__iter__")
        return NotImplemented

class Iterator(Iterable):

    __slots__ = ()

    @abstractmethod
    def __next__(self):
        'Return the next item from the iterator. When exhausted, raise StopIteration'
        raise StopIteration

    def __iter__(self):
        return self

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            return _check_methods(C, '__iter__', '__next__')
        return NotImplemented
```
读源码的好处来了==>**`抽象方法：@abstractmethod（子类必须实现）`，上次漏讲了吧～**

上面说**迭代器肯定可以迭代**，说很抽象，代码太直观了 (继承)：**`class Iterator(Iterable)`**

现在我们来模仿并实现一个`Python`版本的`迭代器`：

In [9]:
% time

# 先搭个空架子
class MyIterator(Iterator):
    def __next__(self):
        pass

class MyArray(Iterable):
    def __iter__(self):
        return MyIterator() # 返回一个迭代器

def main():
    # 可迭代 True
    print(isinstance(MyArray(), Iterable))
    # 迭代器也是可迭代的 True
    print(isinstance(MyIterator(), Iterable))
    # 是迭代器 True
    print(isinstance(MyIterator(), Iterator))

if __name__ == '__main__':
    main()

Wall time: 0 ns
True
True
True


In [10]:
% time

# 把迭代器简化合并
class MyIterator(Iterator):
    def __next__(self):
        pass

    def __iter__(self):
        return self # 返回一个迭代器(现在就是它自己了)

def main():
    print(isinstance(MyIterator(), Iterable))
    print(isinstance(MyIterator(), Iterator))

if __name__ == '__main__':
    main()

Wall time: 0 ns
True
True


In [11]:
% time

# 马上进入正题了，先回顾一下Fibona
def fibona(n):
    a, b = 0, 1
    for i in range(n):
        a, b = b, a+b
        print(a)

# 获取10个斐波拉契数列
fibona(10)

Wall time: 0 ns
1
1
2
3
5
8
13
21
34
55


In [12]:
% time

# 改造成迭代器
from collections.abc import Iterable, Iterator

class FibonaIterator(Iterator):
    def __init__(self, n):
        self.__a = 0
        self.__b = 1
        self.__n = n  # 获取多少个
        self.__index = 0  # 当前索引

    def __next__(self):
        if self.__index < self.__n:
            self.__index += 1
            # 生成下一波
            self.__a, self.__b = self.__b, self.__a + self.__b
            return self.__a
        else:
            raise StopIteration # for循环结束条件

def main():
    print(FibonaIterator(10))
    for i in FibonaIterator(10):
        print(i)

if __name__ == "__main__":
    main()

Wall time: 0 ns
1
1
2
3
5
8
13
21
34
55


### 3.生成器

往期文章：<a href="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.Python生成器" target="_blank">https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.Python生成器</a>

生成器是啥？看源码就秒懂了：(**迭代器的基础上再封装**)
```py
class Generator(Iterator):
    __slots__ = ()

    def __next__(self):
        """从生成器返回下一个item，结束的时候抛出 StopIteration"""
        return self.send(None)

    @abstractmethod
    def send(self, value):
        """将值发送到生成器。返回下一个产生的值或抛出StopIteration"""
        raise StopIteration

    @abstractmethod
    def throw(self, typ, val=None, tb=None):
        """在生成器中引发异常。返回下一个产生的值或抛出StopIteration"""
        if val is None:
            if tb is None:
                raise typ
            val = typ()
        if tb is not None:
            val = val.with_traceback(tb)
        raise val

    # 现在知道之前close后为啥没异常了吧～
    def close(self):
        """屏蔽异常"""
        try:
            self.throw(GeneratorExit)
        except (GeneratorExit, StopIteration):
            pass
        else:
            raise RuntimeError("generator ignored GeneratorExit")

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Generator:
            return _check_methods(C, '__iter__', '__next__',
                                  'send', 'throw', 'close')
        return NotImplemented
```
迭代器的基础上再封装了两个抽象方法`send`、`throw`和屏蔽异常的方法`close`

现在用生成器的方式改写下斐波拉契数列：（列表推导式改成小括号是最简单的一种生成器）

In [13]:
% time

# 代码瞬间就简洁了
def fibona(n):
    a = 0
    b = 1
    for _ in range(n):
        a, b = b, a + b
        yield a # 加个yiel就变成生成器了

def main():
    print(fibona(10))
    for i in fibona(10):
        print(i)

if __name__ == "__main__":
    main()

Wall time: 0 ns
<generator object fibona at 0x000001CAFFD1AC00>
1
1
2
3
5
8
13
21
34
55


注意下这几点：
1. <a herf="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.3.扩展之～send(msg)方法：" target="_blank">generator刚启动的时候，要么 next()，要么 send(None)，不然会引发：</a>
    - `TypeError: can't send non-None value to a just-started generator`
2. <a herf="https://www.cnblogs.com/dotnetcrazy/p/9278573.html#5.4.扩展之～return和break的说明" target="_blank">在一个generator函数中，遇到return或者break则直接抛出StopIteration终止迭代</a>
    - 如果没有则默认执行至函数完毕
3. 如果想要拿到返回值，必须捕获`StopIteration`错误，返回值包含在`StopIteration`的`value`中

```py
def test_send(n):
    for i in range(n):
        if i==2:
            return "i==2"
        yield i

g = test_send(5)

while True:
    try:
        tmp = next(g)
        print(tmp)
    except StopIteration as ex:
        print(ex.value)
        break
```
输出：
```
0
1
i==2
```

其他的也没什么好说的了，读完源码再看看之前讲的内容`别有一番滋味在心头`哦～

## 3.2.概念篇

上集回顾：<a href="https://mp.weixin.qq.com/s/jWRBHi_ZNDBxOXElgAk86w" target="_blank">网络：静态服务器+压测</a>

### 1.同步与异步

> 同步是指一个任务的完成需要依赖另外一个任务时，只有等待被依赖的任务完成后，依赖的任务才能算完成。

> 异步是指不需要等待被依赖的任务完成，只是通知被依赖的任务要完成什么工作。然后继续执行下面代码逻辑，只要自己完成了整个任务就算完成了（异步一般使用状态、通知和回调）

PS：**项目里面一般是这样的**：（个人经验）
1. 同步架构：一般都是和钱相关的需求，需要实时返回的业务
2. 异步架构：更多是对写要求比较高时的场景（同步变异步）
    - 读一般都是实时返回，代码一般都是`await xxx()`
3. 想象个情景就清楚了：
    - 异步：现在用户写了篇文章，可以异步操作，就算没真正写到数据库也可以返回：发表成功（大不了失败提示一下）
    - 同步：用户获取订单信息，你如果异步就会这样了：提示下获取成功，然后一片空白...用户不卸载就怪了...

### 2.阻塞与非阻塞

> 阻塞是指调用结果返回之前，当前线程会被挂起，一直处于等待消息通知，不能够执行其他业务（大部分代码都是这样的）

> 非阻塞是指在不能立刻得到结果之前，该函数不会阻塞当前线程，而会立刻返回（继续执行下面代码，或者重试机制走起）

PS：**项目里面重试机制为啥一般都是3次？**
1. 第一次重试，两台PC挂了也是有可能的
2. 第二次重试，负载均衡分配的三台机器同时挂的可能性不是很大，这时候就有可能是网络有点拥堵了
3. 最后一次重试，再失败就没意义了，日记写起来，再重试网络负担就加大了，得不偿失了

### 3.五种IO模型

对于一次IO访问，数据会先被拷贝到内核的缓冲区中，然后才会从内核的缓冲区拷贝到应用程序的地址空间。需要经历两个阶段：
1. 准备数据
2. 将数据从内核缓冲区拷贝到进程地址空间

由于存在这两个阶段，Linux产生了下面五种IO模型（`以socket为例`）
1. 阻塞式IO：
    - 当用户进程调用了`recvfrom`等阻塞方法时，内核进入IO的第1个阶段：准备数据（内核需要等待足够的数据再拷贝）这个过程需要等待，用户进程会被阻塞，等内核将数据准备好，然后拷贝到用户地址空间，内核返回结果，用户进程才从阻塞态进入就绪态
    - Linux中默认情况下所有的socket都是阻塞的
2. 非阻塞式IO：
    - 当用户进程发出read操作时，如果`kernel`中的数据还没有准备好，那么它并不会`block`用户进程，而是立刻返回一个`error`。
    - 用户进程判断结果是一个`error`时，它就知道数据还没有准备好，于是它可以再次发送read操作
    - 一旦`kernel`中的数据准备好了，并且又再次收到了用户进程的`system call`，那么它马上就将数据拷贝到了用户内存，然后返回
    - 非阻塞IO模式下用户进程需要不断地询问内核的数据准备好了没有
3. **IO多路复用**：
    - 通过一种机制，一个进程可以监视多个文件描述符（套接字描述符）一旦某个文件描述符就绪（一般是读就绪或者写就绪），能够通知程序进行相应的读写操作（这样就不需要每个用户进程不断的询问内核数据准备好了没）
    - 常用的IO多路复用方式有`select`、`poll`和`epoll`
4. 信号驱动IO：（之前我们讲进程先导篇的时候说过）
    - 内核文件描述符就绪后，通过信号通知用户进程，用户进程再通过系统调用读取数据。
    - 此方式属于同步IO（实际读取数据到用户进程缓存的工作仍然是由用户进程自己负责的）
5. **异步IO**（`POSIX`的`aio_`系列函数）
    - 用户进程发起read操作之后，立刻就可以开始去做其它的事。内核收到一个异步`IO read`之后，会立刻返回，不会阻塞用户进程。
    - 内核会等待数据准备完成，然后将数据拷贝到用户内存，当这一切都完成之后，内核会给用户进程发送一个`signal`告诉它read操作完成了

### 4.Unix图示

贴一下Unix编程里面的图：

<center>非阻塞IO</center>

![2.非阻塞IO.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181210212858009-948984805.png)

<center>IO复用</center>

![3.IO复用.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181210212908314-1267377747.png)

<center>信号IO</center>

![4.信号IO.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181210212934040-13536334.png)

<center>异步AIO</center>

![5.异步AIO.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181210212944334-1184572641.png)

## 3.3.IO多路复用

开始之前咱们通过非阻塞IO引入一下：（来个简单例子`socket.setblocking(False)`)
```py
import time
import socket

def select(socket_addr_list):
    for client_socket, client_addr in socket_addr_list:
        try:
            data = client_socket.recv(2048)
            if data:
                print(f"[来自{client_addr}的消息：]\n")
                print(data.decode("utf-8"))
                client_socket.send(
                    b"HTTP/1.1 200 ok\r\nContent-Type: text/html;charset=utf-8\r\n\r\n<h1>Web Server Test</h1>"
                )
            else:
                # 没有消息是触发异常，空消息是断开连接
                client_socket.close()  # 关闭客户端连接
                socket_addr_list.remove((client_socket, client_addr))
                print(f"[客户端{client_addr}已断开连接，当前连接数：{len(socket_addr_list)}]")
        except Exception:
            pass

def main():
    # 存放客户端集合
    socket_addr_list = list()

    with socket.socket() as tcp_server:
        # 防止端口绑定的设置
        tcp_server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        tcp_server.bind(('', 8080))
        tcp_server.listen()
        tcp_server.setblocking(False)  # 服务端非阻塞
        while True:
            try:
                client_socket, client_addr = tcp_server.accept()
                client_socket.setblocking(False)  # 客户端非阻塞
                socket_addr_list.append((client_socket, client_addr))
            except Exception:
                pass
            else:
                print(f"[来自{client_addr}的连接，当前连接数：{len(socket_addr_list)}]")
            # 防止客户端断开后出错
            if socket_addr_list:
                # 轮询查看客户端有没有消息
                select(socket_addr_list)  # 引用传参
                time.sleep(0.01)

if __name__ == "__main__":
    main()
```
输出：
![6.nowait.gif](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181210110004422-72437374.gif)

可以思考下：
1. 为什么Server也要设置为非阻塞？
    - PS：一个线程里面只能有一个死循环，现在程序需要两个死循环，so ==> 放一起咯
2. 断开连接怎么判断？
    - PS：没有消息是触发异常，空消息是断开连接
3. client_socket为什么不用dict存放？
    - PS：dict在循环的过程中，del会引发异常

### 1.Select

select和上面的有点类似，就是轮询的过程交给了操作系统：
> kernel会“监视”所有select负责的socket，当任何一个socket中的数据准备好了，select就会返回。这个时候用户进程再调用read操作，将数据从kernel拷贝到用户进程

来个和上面等同的案例：
```py
import select
import socket

def main():
    with socket.socket() as tcp_server:
        tcp_server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        tcp_server.bind(('', 8080))
        tcp_server.listen()
        socket_info_dict = dict()
        socket_list = [tcp_server]  # 监测列表
        while True:
            # 劣势：select列表数量有限制
            read_list, write_list, error_list = select.select(
                socket_list, [], [])
            for item in read_list:
                # 服务端迎接新的连接
                if item == tcp_server:
                    client_socket, client_address = item.accept()
                    socket_list.append(client_socket)
                    socket_info_dict[client_socket] = client_address
                    print(f"[{client_address}已连接，当前连接数：{len(socket_list)-1}]")
                # 客户端发来
                else:
                    data = item.recv(2048)
                    if data:
                        print(data.decode("utf-8"))
                        item.send(
                            b"HTTP/1.1 200 ok\r\nContent-Type: text/html;charset=utf-8\r\n\r\n<h1>Web Server Test</h1>"
                        )
                    else:
                        item.close()
                        socket_list.remove(item)
                        info = socket_info_dict[item]
                        print(f"[{info}已断开，当前连接数：{len(socket_list)-1}]")

if __name__ == "__main__":
    main()
```
**输出和上面一样**

扩展说明：
> select 函数监视的文件描述符分3类，分别是`writefds`、`readfds`、和`exceptfds`。调用后select函数会阻塞，直到有描述符就绪函数返回（**有数据可读、可写、或者有except**）或者超时（timeout指定等待时间，如果立即返回设为null即可）

> select的一个缺点在于单个进程能够监视的文件描述符的数量存在最大限制，在Linux上一般为1024（64位=>2048）

**然后Poll就出现了，就是把上限给去掉了，本质并没变，还是使用的`轮询`**

### 2.EPoll

> epoll在内核2.6中提出（Linux独有），使用一个文件描述符管理多个描述符，将用户关心的文件描述符的事件存放到内核的一个事件表中，采用监听回调的机制，这样在用户空间和内核空间的copy只需一次，避免再次遍历就绪的文件描述符列表

先来看个案例吧：（输出和上面一样）
```py
import socket
import select

def main():
    with socket.socket() as tcp_server:
        tcp_server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        tcp_server.bind(('', 8080))
        tcp_server.listen()

        # epoll是linux独有的
        epoll = select.epoll()
        # tcp_server注册到epoll中
        epoll.register(tcp_server.fileno(), select.EPOLLIN | select.EPOLLET)

        # key-value
        fd_socket_dict = dict()

        # 回调需要自己处理
        while True:
            # 返回可读写的socket fd 集合
            poll_list = epoll.poll()
            for fd, event in poll_list:
                # 服务器的socket
                if fd == tcp_server.fileno():
                    client_socket, client_addr = tcp_server.accept()
                    fd = client_socket.fileno()
                    fd_socket_dict[fd] = (client_socket, client_addr)
                    # 把客户端注册进epoll中
                    epoll.register(fd, select.EPOLLIN | select.EPOLLET)
                else:  # 客户端
                    client_socket, client_addr = fd_socket_dict[fd]
                    data = client_socket.recv(2048)
                    print(
                        f"[来自{client_addr}的消息，当前连接数：{len(fd_socket_dict)}]\n")
                    if data:
                        print(data.decode("utf-8"))
                        client_socket.send(
                            b"HTTP/1.1 200 ok\r\nContent-Type: text/html;charset=utf-8\r\n\r\n<h1>Web Server Test</h1>"
                        )
                    else:
                        del fd_socket_dict[fd]
                        print(
                            f"[{client_addr}已离线，当前连接数：{len(fd_socket_dict)}]\n"
                        )
                        # 从epoll中注销
                        epoll.unregister(fd)
                        client_socket.close()

if __name__ == "__main__":
    main()
```

扩展：**epoll的两种工作模式**
> LT（level trigger，水平触发）模式：当epoll_wait检测到描述符就绪，将此事件通知应用程序，应用程序可以不立即处理该事件。下次调用epoll_wait时，会再次响应应用程序并通知此事件。LT模式是默认的工作模式。
> LT模式同时支持阻塞和非阻塞socket。

> ET（edge trigger，边缘触发）模式：当epoll_wait检测到描述符就绪，将此事件通知应用程序，应用程序必须立即处理该事件。如果不处理，下次调用epoll_wait时，不会再次响应应用程序并通知此事件。
> ET是高速工作方式，只支持非阻塞socket（ET模式减少了epoll事件被重复触发的次数，因此效率要比LT模式高）

**Code提炼一下**：
1. 实例化对象：`epoll = select.epoll()`
2. 注册对象：`epoll.register(tcp_server.fileno(), select.EPOLLIN | select.EPOLLET)`
3. 注销对象：`epoll.unregister(fd)`

PS：`epoll`不一定比`Select`性能高，一般都是分场景的：
1. 高并发下，连接活跃度不高时：epoll比Select性能高（eg：web请求，页面随时关闭）
2. 并发不高，连接活跃度比较高：Select更合适（eg：小游戏）
3. **Select是win和linux通用的，而epoll只有linux有**

其实IO多路复用还有一个`kqueue`，和`epoll`类似，下面的通用写法中有包含

---

### 3.通用写法（`Selector`）

一般来说：**Linux下使用epoll，Win下使用select**（IO多路复用会这个通用的即可）

先看看Python源代码：
```py
# 选择级别：epoll|kqueue|devpoll > poll > select
if 'KqueueSelector' in globals():
    DefaultSelector = KqueueSelector
elif 'EpollSelector' in globals():
    DefaultSelector = EpollSelector
elif 'DevpollSelector' in globals():
    DefaultSelector = DevpollSelector
elif 'PollSelector' in globals():
    DefaultSelector = PollSelector
else:
    DefaultSelector = SelectSelector
```

**实战案例**：(可读和可写可以不分开)
```py
import socket
import selectors

# Linux下使用epoll，Win下使用select
Selector = selectors.DefaultSelector()

class Task(object):
    def __init__(self):
        # 存放客户端fd和socket键值对
        self.fd_socket_dict = dict()

    def run(self):
        self.server = socket.socket()
        self.server.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server.bind(('', 8080))
        self.server.listen()
        # 把Server注册到epoll
        Selector.register(self.server.fileno(), selectors.EVENT_READ,
                          self.connected)

    def connected(self, key):
        """客户端连接时处理"""
        client_socket, client_address = self.server.accept()
        fd = client_socket.fileno()
        self.fd_socket_dict[fd] = (client_socket, client_address)
        # 注册一个客户端读的事件（服务端去读消息）
        Selector.register(fd, selectors.EVENT_READ, self.call_back_reads)
        print(f"{client_address}已连接，当前连接数：{len(self.fd_socket_dict)}")

    def call_back_reads(self, key):
        """客户端可读时处理"""
        # 一个fd只能注册一次，监测可写的时候需要把可读给注销
        Selector.unregister(key.fd)
        client_socket, client_address = self.fd_socket_dict[key.fd]
        print(f"[来自{client_address}的消息:]\n")
        data = client_socket.recv(2048)
        if data:
            print(data.decode("utf-8"))
            # 注册一个客户端写的事件（服务端去发消息）
            Selector.register(key.fd, selectors.EVENT_WRITE,
                              self.call_back_writes)
        else:
            client_socket.close()
            del self.fd_socket_dict[key.fd]
            print(f"{client_address}已断开，当前连接数：{len(self.fd_socket_dict)}")

    def call_back_writes(self, key):
        """客户端可写时处理"""
        Selector.unregister(key.fd)
        client_socket, client_address = self.fd_socket_dict[key.fd]
        client_socket.send(b"ok")
        Selector.register(key.fd, selectors.EVENT_READ, self.call_back_reads)

def main():
    t = Task()
    t.run()
    while True:
        ready = Selector.select()
        for key, obj in ready:
            # 需要自己回调
            call_back = key.data
            call_back(key)

if __name__ == "__main__":
    main()
```

**Code提炼一下**：
1. 实例化对象：`Selector = selectors.DefaultSelector()`
2. 注册对象：
    - `Selector.register(server.fileno(), selectors.EVENT_READ, call_back)`
    - `Selector.register(server.fileno(), selectors.EVENT_WRITE, call_back)`
3. 注销对象：`Selector.unregister(key.fd)`
4. 注意一下：**一个fd只能注册一次，监测可写的时候需要把可读给注销（反之一样）**

业余拓展：
```
select, iocp, epoll,kqueue及各种I/O复用机制
https://blog.csdn.net/shallwake/article/details/5265287

kqueue用法简介
http://www.cnblogs.com/luminocean/p/5631336.html
```

## 3.4.协程引入

### 1.yield from

我们经常有这样的需求：`读取两个分表的数据列表，然后合并之后进行一些处理`

平时可以借用`itertools.chain`来遍历：
```py
# https://docs.python.org/3/library/itertools.html#itertools.chain
import itertools

def main():
    # 模拟分表后的两个查询结果
    user1 = ["小张", "小明"]
    user2 = ["小潘", "小周"]
    # dict只能遍历key（这种情况需要自己封装合并方法并处理下）
    user3 = {"name": "test1", "name1": "test2"}
    
    # 需求：合并并遍历
    for item in itertools.chain(user1, user2, user3):
        print(item)

if __name__ == '__main__':
    main()
```
输出：
```
小张
小明
小潘
小周
name
name1
```

它的内部实现其实是这样的：（`相当于两层遍历，用yield返回`）
```py
def my_chain(*args, **kwargs):
    for items in args:
        for item in items:
            yield item

def main():
    # 模拟分表后的两个查询结果
    user1 = ["小张", "小明"]
    user2 = ["小潘", "小周"]
    # dict只能遍历key（这种情况需要自己封装合并方法并处理下）
    user3 = {"name": "test1", "name1": "test2"}
    
    # 需求：合并并遍历
    for item in my_chain(user1, user2, user3):
        print(item)

if __name__ == '__main__':
    main()
```

然后`Python3.3`之后语法再一步简化（**`yield from iterable对象`**）

```py
def my_chain(*args, **kwargs):
    for items in args:
        yield from items

def main():
    # 模拟分表后的两个查询结果
    user1 = ["小张", "小明"]
    user2 = ["小潘", "小周"]
    
    # 需求：合并并遍历
    for item in my_chain(user1, user2):
        print(item)

if __name__ == '__main__':
    main()
```
输出：
```
小张
小明
小潘
小周
test1
test2
```

#### 扩展（可忽略）

其实知道了内部实现，很容易就写上一段应对的处理：
```py
def my_chain(*args, **kwargs):
    for my_iterable in args:
        # 如果是字典类型就返回value
        if isinstance(my_iterable, dict):
            my_iterable = my_iterable.values()
        for item in my_iterable:
            yield item

def main():
    # 模拟分表后的两个查询结果
    user1 = ["小张", "小明"]
    user2 = ["小潘", "小周"]
    # dict只能遍历key（这种情况需要自己封装合并方法并处理下）
    user3 = {"name": "test1", "name1": "test2"}
    # 需求：合并并遍历
    for item in my_chain(user1, user2, user3):
        print(item)

if __name__ == '__main__':
    main()
```
输出：
```
小张
小明
小潘
小周
test1
test2
```
#### 扩展的正确处理

PS：一般不会这么干的，一般都是`[{},{}]`遍历并处理：
```py
import itertools

def main():
    # 模拟分表后的两个查询结果
    user1 = [{"name": "小张"}, {"name": "小明"}]
    user2 = [{"name": "小潘"}, {"name": "小周"}]
    user3 = [{"name": "test1"}, {"name": "test2"}]
    # 需求：合并并遍历
    for item in itertools.chain(user1, user2, user3):
        # 一般都是直接在这里进行处理
        for key, value in item.items():
            print(value)

if __name__ == '__main__':
    main()
```

### 1.yield版协程

协程的目的其实很简单：**像写同步代码那样实现异步编程**

先看个需求：**生成绘图的数据（`max,min,avg`）**

比如说原来数据是这样的：
```py
products = [{
    "id": 2344,
    "title": "御泥坊补水面膜",
    "price": [89, 76, 120, 99]
}, {
    "id": 2345,
    "title": "御泥坊火山泥面膜",
    "price": [30, 56, 70, 89]
}]
```
处理之后：
```py
new_products = [{
    "id": 2344,
    "title": "御泥坊补水面膜",
    "price": [89, 76, 120, 99],
    "max": 120,
    "min": 76,
    "avg": 96.0
},
{
    "id": 2345,
    "title": "御泥坊火山泥面膜",
    "price": [30, 56, 70, 89],
    "max": 89,
    "min": 30,
    "avg": 61.25
}]
```

处理过的数据一般用来画图，实际效果类似于：
![1.需求.png](https://img2018.cnblogs.com/blog/1127869/201812/1127869-20181211133211122-601142215.png)

如果不借助协程，我们一般这么处理：（数据库获取过程省略）

In [14]:
# 生成新的dict数据
def get_new_item(item):
    prices = item["price"]
    item["avg"] = sum(prices) / len(prices)
    item["max"] = max(prices)
    item["min"] = min(prices)
    return item

def get_new_data(data):
    newdata = []
    for item in data:
        new_item = get_new_item(item)
        # print(new_item) # 处理后的新dict
        newdata.append(new_item)
    return newdata

def main():
    # 需求：生成绘图的数据（max,min,avg）
    products = [{
        "id": 2344,
        "title": "御泥坊补水面膜",
        "price": [89, 76, 120, 99]
    }, {
        "id": 2345,
        "title": "御泥坊火山泥面膜",
        "price": [30, 56, 70, 89]
    }]

    new_products = get_new_data(products)
    print(new_products)

if __name__ == "__main__":
    main()

[{'id': 2344, 'title': '御泥坊补水面膜', 'price': [89, 76, 120, 99], 'avg': 96.0, 'max': 120, 'min': 76}, {'id': 2345, 'title': '御泥坊火山泥面膜', 'price': [30, 56, 70, 89], 'avg': 61.25, 'max': 89, 'min': 30}]


改成yield版的协程也很方便，基本上代码没有变，也不用像IO多路复用那样来回的回调

In [15]:
# 生成新的dict数据
def get_new_item(item):
    prices = item["price"]
    item["avg"] = sum(prices) / len(prices)
    item["max"] = max(prices)
    item["min"] = min(prices)
    yield item

def get_new_data(data):
    for item in data:
        yield from get_new_item(item)

def main():
    # 需求：生成绘图的数据（max,min,avg）
    products = [{
        "id": 2344,
        "title": "御泥坊补水面膜",
        "price": [89, 76, 120, 99]
    }, {
        "id": 2345,
        "title": "御泥坊火山泥面膜",
        "price": [30, 56, 70, 89]
    }]
    new_products = list()
    # 如果需要返回值就捕获StopIteration异常
    for item in get_new_data(products):
        new_products.append(item)
    print(new_products)

if __name__ == "__main__":
    main()

[{'id': 2344, 'title': '御泥坊补水面膜', 'price': [89, 76, 120, 99], 'avg': 96.0, 'max': 120, 'min': 76}, {'id': 2345, 'title': '御泥坊火山泥面膜', 'price': [30, 56, 70, 89], 'avg': 61.25, 'max': 89, 'min': 30}]


简单解析一下：（用`yield from`的目的就是为了引出等会说的`async/await`）

**`yield from`（委托生成器`get_new_data`）的好处就是让调用方（`main`）和`yield`子生成器(`get_new_item`)直接建立一个双向通道**

你也可以把`yield from`当作一个中介(**如果不理解就把`yield from`想象成`await`就容易理解了**)，本质就是下面代码：

In [16]:
# 生成新的数据
def get_new_data(data):
    for item in data:
        prices = item["price"]
        item["avg"] = sum(prices) / len(prices)
        item["max"] = max(prices)
        item["min"] = min(prices)
        yield item


def main():
    # 需求：生成绘图的数据（max,min,avg）
    products = [{
        "id": 2344,
        "title": "御泥坊补水面膜",
        "price": [89, 76, 120, 99]
    }, {
        "id": 2345,
        "title": "御泥坊火山泥面膜",
        "price": [30, 56, 70, 89]
    }]
    new_products = list()
    for item in get_new_data(products):
        new_products.append(item)
    print(new_products)


if __name__ == "__main__":
    main()

[{'id': 2344, 'title': '御泥坊补水面膜', 'price': [89, 76, 120, 99], 'avg': 96.0, 'max': 120, 'min': 76}, {'id': 2345, 'title': '御泥坊火山泥面膜', 'price': [30, 56, 70, 89], 'avg': 61.25, 'max': 89, 'min': 30}]


#### PEP 380（含分析）

`yield from`内部其实在`yield`基础上做了很多事情（比如一些异常的处理），具体可以看看 **<a href="https://www.python.org/dev/peps/pep-0380/" target="_blank">PEP 380</a>**

先提炼一个`简版`的：
```py
# 正常调用
RESULT = yield from EXPR

# _i：子生成器（也是个迭代器）
# _y：子生成器生产的值
# _r：yield from 表达式最终结果
# _s：调用方通过send发送的值
# _e：异常对象

# 内部原理
_i = iter(EXPR) # EXPR是一个可迭代对象，_i是子生成器
try:
    # 第一次不能send值，只能next() or send(None)，并把产生的值放到_y中
    _y = next(_i)
except StopIteration as _e:
    # 如果子生成器直接就return了，那就会抛出异常，通过value可以拿到子生成器的返回值
    _r = _e.value
else:
    # 尝试进行循环（调用方和子生成器交互过程），yield from这个生成器会阻塞（委托生成器）
    while 1:
        # 这时候子生成器已经和调用方建立了双向通道，在等待调用方send(value)，把这个值保存在_s中
        _s = yield _y # 这边还会进行一系列异常处理，我先删掉，等会看
        try:
            # 如果send(None)，那么继续next遍历
            if _s is None:
                _y = next(_i) # 把子生成器结果放到 _y 中
            else:
                _y = _i.send(_s) # 如果调用方send一个值，就转发到子生成器
        except StopIteration as _e:
            _r = _e.value # 如果子生成器遍历完了，就把返回值给_r
            break
RESULT = _r # 最终的返回值（yield from 最终的返回值）
```

现在再来看`完整版`压力就没有那么大了：

```py
# 正常调用
RESULT = yield from EXPR

# _i：子生成器（也是个迭代器）
# _y：子生成器生产的值
# _r：yield from 表达式最终结果
# _s：调用方通过send发送的值
# _e：异常对象

# 内部原理
_i = iter(EXPR) # EXPR是一个可迭代对象，_i是子生成器
try:
    # 第一次不能send值，只能next() or send(None)，并把产生的值放到_y中
    _y = next(_i)
except StopIteration as _e:
    # 如果子生成器直接就return了，那就会抛出异常，通过value可以拿到子生成器的返回值
    _r = _e.value
else:
    # 尝试进行循环（调用方和子生成器交互过程），yield from这个生成器会阻塞（委托生成器）
    while 1:
        try:
            # 这时候子生成器已经和调用方建立了双向通道，在等待调用方send(value)，把这个值保存在_s中
            _s = yield _y
        
        # 【现在补全】有这么几种情况需要处理
        # 1.子生成器可能只是一个迭代器，并不能作为协程的生成器（不支持throw和close）
        # 2.子生成器虽然支持了throw和close，但在子生成器内部两种方法都会抛出异常
        # 3.调用法调用了gen.throw()，想让子生成器自己抛异常
        # 这时候就要处理 gen.close() 和 gen.throw()的情况
        
        # 生成器close()异常的处理
        except GeneratorExit as _e:
            try:
                _m = _i.close
            except AttributeError:
                pass # 屏蔽close的异常
            else:
                _m()
            raise _e # 上抛异常
        # 生成器throw()异常的处理
        except BaseException as _e:
            _x = sys.exc_info()
            try:
                _m = _i.throw
            except AttributeError:
                raise _e
            else:
                try:
                    _y = _m(*_x)
                except StopIteration as _e:
                    _r = _e.value
                    break
        else:
            try:
                # 如果send(None)，那么继续next遍历
                if _s is None:
                    _y = next(_i) # 把子生成器结果放到 _y 中
                else:
                    _y = _i.send(_s) # 如果调用方send一个值，就转发到子生成器
            except StopIteration as _e:
                _r = _e.value # 如果子生成器遍历完了，就把返回值给_r
                break
RESULT = _r # 最终的返回值（yield from 最终的返回值）
```

### 2.async/await

把上面的原生代码用`async和await`改装一下：(**协程的目的就是像写同步代码一样写异步，这个才算是真做到了**)
```py
import asyncio

# 生成新的dict数据
async def get_new_item(item):
    prices = item["price"]
    item["avg"] = sum(prices) / len(prices)
    item["max"] = max(prices)
    item["min"] = min(prices)
    return item

async def get_new_data(data):
    newdata = []
    for item in data:
        new_item = await get_new_item(item)
        # print(new_item) # 处理后的新dict
        newdata.append(new_item)
    return newdata

def main():
    # 需求：生成绘图的数据（max,min,avg）
    products = [{
        "id": 2344,
        "title": "御泥坊补水面膜",
        "price": [89, 76, 120, 99]
    }, {
        "id": 2345,
        "title": "御泥坊火山泥面膜",
        "price": [30, 56, 70, 89]
    }]
    
    # python 3.7
    new_products = asyncio.run(get_new_data(products))
    print(new_products)

if __name__ == "__main__":
    main()
```
输出：（是不是很原生代码没啥区别？）
```
[{'id': 2344, 'title': '御泥坊补水面膜', 'price': [89, 76, 120, 99], 'avg': 96.0, 'max': 120, 'min': 76}, 
{'id': 2345, 'title': '御泥坊火山泥面膜', 'price': [30, 56, 70, 89], 'avg': 61.25, 'max': 89, 'min': 30}]
```

下级预估：**asyncio**

## 3.5.asyncio

官方文档：<a href="https://docs.python.org/3/library/asyncio.html" target="_blank">https://docs.python.org/3/library/asyncio.html</a>

开发中常见错误：<a href="https://docs.python.org/3/library/asyncio-dev.html" target="_blank">https://docs.python.org/3/library/asyncio-dev.html</a>

代码示例：<https://github.com/lotapp/BaseCode/tree/master/python/5.concurrent/ZCoroutine>

### 3.5.1.上节回顾

上次说了下<a href="https://mp.weixin.qq.com/s/5d701WQG1L8mDOO-bnKXdQ" target="_blank">协程演变过程</a>，这次继续，先接着上次的说：

像`JS`是可以生成器和`async`和`await`混用的，那`Python`呢？（NetCore不可以混用）
```py
import types

# 和生成器完全分开了，不过可以理解为yield from
@types.coroutine
def get_value(value):
    yield value

async def get_name(name):
    # 一系列逻辑处理
    return await get_value(name)

if __name__ == '__main__':
    gen = get_name("小明")
    print(gen.send(None))
# 直接混用会报错：TypeError: object generator can't be used in 'await' expression
```

我们的`async`和`await`虽然和`yield from`不是一个概念，但是可以理解为`yield from`上面这段代码你可以理解为：
```py
import types

def get_value(value):
    yield value

# 这个async和await替换成yield from
def get_name(name):
    # 一系列逻辑处理
    yield from get_value(name)

if __name__ == '__main__':
    gen = get_name("小明")
    print(gen.send(None))
```
PS：**Python默认和NetCore一样，不能直接混用，如果你一定要混用，那么得处理下**（`使用@asyncio.coroutine`也行）

### 3.5.2.asyncio引入

在今天之前，协程我们是这么实现的：`事件循环(loop)`+`回调(驱动生成器)`+`IO多路复用(epoll)`

现在可以通过官方提供的`asyncio`（**可以理解为协程池**）来实现了（第三方还有一个`uvloop`【基于C写的`libuv`库（`nodejs`也是基于这个库）】）

PS：`uvloop`的使用非常简单，只要在获取事件循环前将`asyncio`的事件循环策略设置为`uvloop`的:**`asyncio.set_event_loop_policy(uvloop.EventLoopPolicy())`**

#### 1.简单案例

先看个简单的协程案例：
```py
import types
import asyncio

# 模拟一个耗时操作
async def test():
    print("start...")
    # 不能再使用以前阻塞的暂停了
    await asyncio.sleep(2)
    print("end...")
    return "ok"

if __name__ == '__main__':
    import time
    start_time = time.time()
    
    # # >=python3.4
    # # 返回asyncio的事件循环
    # loop = asyncio.get_event_loop()
    # # 运行事件循环，直到指定的future运行完毕，返回结果
    # result = loop.run_until_complete(test())
    # print(result)
    
    # python3.7
    result = asyncio.run(test())
    print(result)

    print(time.time() - start_time)
```
输出：
```
start...
end...
ok
2.001772403717041
```

简单说下，`asyncio.run`是python3.7才简化出来的语法（类比NetCore的`Task.Run`）看看源码就知道了：
```py
# https://github.com/lotapp/cpython3/blob/master/Lib/asyncio/runners.py
def run(main, *, debug=False):
    # 3.7开始推荐使用"asyncio.get_running_loop()"来获取loop
    # 以前是直接使用"asyncio.get_event_loop()"
    if events._get_running_loop() is not None:
        raise RuntimeError("无法从正在运行的事件循环中调用asyncio.run()")

    if not coroutines.iscoroutine(main):
        raise ValueError("{!r}应该是一个协程".format(main))

    loop = events.new_event_loop() # 创建一个新的事件循环
    try:
        events.set_event_loop(loop)  # 设置事件循环
        loop.set_debug(debug)  # 是否调试运行（默认否）
        return loop.run_until_complete(main)  # 等待运行
    finally:
        try:
            _cancel_all_tasks(loop)  # 取消其他任务
            loop.run_until_complete(loop.shutdown_asyncgens())
        finally:
            events.set_event_loop(None)
            loop.close()
```
新版本其实就是使用了一个新的`loop`去启动`run_until_complete`

PS：`uvloop`也可以这样去使用：获取loop`loop = uvloop.new_event_loop()`再替换原生的loop`asyncio.set_event_loop(loop)`

### 3.5.3.批量任务

#### 1.旧版本实现（不推荐）

```py
import asyncio

# 模拟一个耗时操作
async def test(i):
    print("start...")
    # 不能再使用以前阻塞的暂停了
    await asyncio.sleep(2)
    print("end...")
    return i

if __name__ == '__main__':
    import time

    start_time = time.time()

    # # >=python3.4
    loop = asyncio.get_event_loop()
    # tasks = [asyncio.ensure_future(test(i)) for i in range(10)]
    # 注意：是loop的方法，而不是asyncio的，不然就会引发RuntimeError：no running event loop
    tasks = [loop.create_task(test(i)) for i in range(10)]
    loop.run_until_complete(asyncio.wait(tasks))
    for task in tasks:
        print(task.result())

    print(time.time() - start_time)
```
输出：(tasks替换成这个也一样：`tasks = [asyncio.ensure_future(test(i)) for i in range(10)]`)
```
start...
start...
start...
start...
start...
start...
start...
start...
start...
start...
end...
end...
end...
end...
end...
end...
end...
end...
end...
end...
0
1
2
3
4
5
6
7
8
9
2.028331995010376
```

然后我们再看看这个`asyncio.wait`是个啥：（回顾：<https://www.cnblogs.com/dotnetcrazy/p/9528315.html#wait()说明>）
```
# return_when 这个参数和之前一样
FIRST_COMPLETED = concurrent.futures.FIRST_COMPLETED
FIRST_EXCEPTION = concurrent.futures.FIRST_EXCEPTION
ALL_COMPLETED = concurrent.futures.ALL_COMPLETED

# 和concurrent.futures里面的wait不一样，这边是个协程
async def wait(fs, *, loop=None, timeout=None, return_when=ALL_COMPLETED):
```

**如果你不想大改动，只是平滑过度到新版本，可以使用`asyncio.gather(*tasks)`来替换`asyncio.wait(tasks)`**

#### 1.旧版新用

PS：官方推荐使用`create_task的方式`，然后值得说的就是`asyncio.wait`==>官方准备在未来版本废弃它，所以我推荐大家这样写：
```py
import asyncio

# 模拟一个耗时操作
async def test(i):
    print("start...")
    # 不能再使用以前阻塞的暂停了
    await asyncio.sleep(2)
    print("end...")
    return i

async def main():
    tasks = [test(i) for i in range(10)]
    # await task 可以得到返回值（得到结果或者异常）
    return [await task for task in asyncio.as_completed(tasks)]

if __name__ == '__main__':
    import time

    start_time = time.time()

    # old推荐使用
    loop = asyncio.get_event_loop()
    result_list = loop.run_until_complete(main())
    print(result_list)
    
    print(time.time() - start_time)
```
输出：(PS：用`asyncio.gather(*tasks)`直接替换`asyncio.wait(tasks)`也行)
```
start...
start...
start...
start...
start...
start...
start...
start...
start...
start...
end...
end...
end...
end...
end...
end...
end...
end...
end...
end...
[5, 8, 9, 4, 7, 6, 3, 0, 1, 2]
2.025350332260132
```

**其实理解起来很简单，而且和`NetCore`以及`NodeJS`它们统一了，只要是`await xxx`就返回一个（`结果`|`异常`），`不await`就是一个`task对象`**

#### 2.新版本实现

```py
import asyncio

# 模拟一个耗时操作
async def test(i):
    print("start...")
    await asyncio.sleep(2)
    print("end...")
    return i

async def main():
    tasks = [test(i) for i in range(10)]
    # 给`协程/futures`返回一个future聚合结果
    return await asyncio.gather(*tasks) # 记得加*来解包

if __name__ == '__main__':
    import time

    start_time = time.time()

    # python3.7
    result_list = asyncio.run(main())
    print(result_list)
    
    # 2.0259485244750977
    print(time.time() - start_time)
```
输出：(语法简化太多了，用起来特别简单)
```
start...
start...
start...
start...
start...
start...
start...
start...
start...
start...
end...
end...
end...
end...
end...
end...
end...
end...
end...
end...
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
2.00840163230896
```

关于参数需要加`*`解包的说明 ==> 看看函数定义就秒懂了：
```py
# 给 协程/futures 返回一个future聚合结果
def gather(*coros_or_futures, loop=None, return_exceptions=False):
    pass

# 把协程或者awaitable对象包裹成task
def ensure_future(coro_or_future, *, loop=None):
    pass

# 传入一个协程对象，返回一个task对象
class BaseEventLoop(events.AbstractEventLoop):
    def create_task(self, coro):
        pass
```

### 关于高级和低级API的说明

**asyncio的高级（`high-level`）API一般用于这几个方面**：（开发基本够用了）
1. 并行运行Python协同程序并完全控制它们的执行
2. **网络通信（`IO`）和进程间通信（`IPC`）**
3. **子进程**（`subprocesses`）相关
4. 通过**队列**（`Queue`）分配任务（`Tasks`）
5. **同步**（`synchronize`）并发代码

**低级（`low-level`）API一般这么用**：（事件循环和回调会用下，其他基本不用）
1. 创建和管理**事件循环**，为网络、子进程、信号处理（`Signal`）等提供异步（`asynchronous `）API
2. 为传输使用高效协议
3. 使用`async/await`语法桥接基于**回调**的库和代码

### 3.5.4.回调函数

回调一般不利于代码维护，现在基本上是尽量不用了（异步代码用起来都和同步没多大差别了，回调也就没那么大用处了）

#### 1.回调函数获取返回值

上面说的获取返回值，其实也可以通过回调函数来获取：
```py
# 低级API示例
import asyncio

async def get_html(url):
    print(f"get {url} ing")
    await asyncio.sleep(2)
    return f"<h1>This is a test for {url}</h1>"

def call_back(task):
    print(type(task))
    print(task.result())

if __name__ == "__main__":
    import time
    start_time = time.time()

    urls = [
        "https://www.baidu.com", "https://www.sogou.com",
        "https://www.python.org", "https://www.asp.net"
    ]
    tasks = set()  # 任务集合
    loop = asyncio.get_event_loop()
    for url in urls:
        # task = asyncio.ensure_future(get_html(url))
        task = loop.create_task(get_html(url))
        # 设置回调函数
        task.add_done_callback(call_back)
        # 添加到任务集合中
        tasks.add(task)
    # 批量执行
    loop.run_until_complete(asyncio.gather(*tasks))

    print(time.time() - start_time)
```
输出：（**`task.add_done_callback(回调函数)`**）
```
get https://www.baidu.com ing
get https://www.sogou.com ing
get https://www.python.org ing
get https://www.asp.net ing
<class '_asyncio.Task'>
<h1>This is a test for https://www.baidu.com</h1>
<class '_asyncio.Task'>
<h1>This is a test for https://www.python.org</h1>
<class '_asyncio.Task'>
<h1>This is a test for https://www.sogou.com</h1>
<class '_asyncio.Task'>
<h1>This is a test for https://www.asp.net</h1>
2.0168468952178955
```

#### 2.回调函数传参扩展

实例：
```py
import asyncio
import functools

async def get_html(url):
    await asyncio.sleep(2)
    return "This is a test for"

# 注意一个东西：通过偏函数传过来的参数在最前面
def call_back(url, task):
    # do something
    print(type(task))
    print(task.result(), url)

if __name__ == "__main__":
    import time
    start_time = time.time()

    urls = [
        "https://www.baidu.com", "https://www.sogou.com",
        "https://www.python.org", "https://www.asp.net"
    ]
    tasks = set()  # 任务集合
    loop = asyncio.get_event_loop()
    for url in urls:
        # task = asyncio.ensure_future(get_html(url))
        task = loop.create_task(get_html(url))
        # 设置回调函数 （不支持传参数，我们就利用偏函数来传递）
        task.add_done_callback(functools.partial(call_back, url))
        # 添加到任务集合中
        tasks.add(task)
    # 批量执行
    loop.run_until_complete(asyncio.gather(*tasks))

    print(time.time() - start_time)
```
输出：(**PS：通过偏函数传过来的参数在最前面**)
```
<class '_asyncio.Task'>
This is a test for https://www.baidu.com
<class '_asyncio.Task'>
This is a test for https://www.python.org
<class '_asyncio.Task'>
This is a test for https://www.sogou.com
<class '_asyncio.Task'>
This is a test for https://www.asp.net
2.0167236328125
```

### 3.5.5.异常相关

之前说的`await task`可能得到结果也可能得到异常有些人可能还不明白 ==> 其实你把他看出同步代码（PS：协程的目的就是**像写同步代码一样进行异步编程**）就好理解了，函数执行要么得到结果要么得到返回值

看个异常的案例：
```py
import asyncio

async def get_html(url):
    print(f"get {url} ing")
    if url == "https://www.asp.net":
        raise Exception("Exception is over")
    await asyncio.sleep(2)
    return f"<h1>This is a test for {url}</h1>"

async def main():
    urls = [
        "https://www.baidu.com", "https://www.asp.net",
        "https://www.python.org", "https://www.sogou.com"
    ]
    tasks = [get_html(url) for url in urls]
    return await asyncio.gather(*tasks)

if __name__ == "__main__":
    import time
    start_time = time.time()

    try:
        asyncio.run(main())
    except Exception as ex:
        print(ex)

    print(time.time() - start_time)

```
输出：(**和同步代码没差别，可能出异常的部分加个异常捕获即可**)
```
get https://www.baidu.com ing
get https://www.asp.net ing
get https://www.python.org ing
get https://www.sogou.com ing
Exception is over
0.008000373840332031
```

再一眼旧版怎么用：（PS：基本差不多，下次全部用新用法了）
```py
import asyncio

async def get_html(url):
    print(f"get {url} ing")
    if url == "https://www.asp.net":
        raise Exception("Exception is over")
    await asyncio.sleep(2)
    return f"<h1>This is a test for {url}</h1>"

async def main():
    urls = [
        "https://www.baidu.com", "https://www.asp.net",
        "https://www.python.org", "https://www.sogou.com"
    ]
    tasks = set()  # 任务集合
    tasks = [get_html(url) for url in urls]
    return await asyncio.gather(*tasks)

if __name__ == "__main__":
    import time
    start_time = time.time()

    loop = asyncio.get_event_loop()
    try:
        # 批量执行
        loop.run_until_complete(main())
    except Exception as ex:
        print(ex)

    print(time.time() - start_time)
```

### 常见异常

**Python3调试过程中的常见异常**：<https://www.cnblogs.com/dotnetcrazy/p/9192089.html>

#### asyncio中常见异常

官方文档：`https://docs.python.org/3/library/asyncio-exceptions.html`
1. **`asyncio.TimeoutError(Exception.Error)`：**
    - 任务超时引发的异常
2. **`asyncio.CancelledError(Exception.Error)`：**
    - 任务取消引发的异常
3. `asyncio.InvalidStateError(Exception.Error)`：
    - `Task/Future`内部状态无效引发
4. `asyncio.IncompleteReadError(Exception.Error)`：读取未完成引发的错误:
    - 不完整: 在到达流结束之前读取字节字符串（读取了不完整的字符串就转换了）
    - 不清楚读多少: 预期读取的字节总数未知
5. `asyncio.LimitOverrunError(Exception)`：
    - 超出缓冲区引发的异常
6. `asyncio.SendfileNotAvailableError(Exception.ReferenceError.RuntimeError)`：
    - 系统调用不适用于给定的套接字或文件类型（系统调用类型不匹配导致的）

#### Python常见异常

有些异常官方没有写进去，我补了一些常用的异常：`https://docs.python.org/3/library/exceptions.html`

**`BaseException`**
- `SystemExit`：`sys.exit()`引发的异常（目的：让Python解释器退出）
- `KeyboardInterrupt`：用户Ctrl+C终止程序引发的异常
- **`GeneratorExit`**：生成器或者协程关闭的时候产生的异常（**特别注意**）
- **`Exception`**：所有内置异常（非系统退出）或者用户定义异常的基类
    - **`asyncio.Error`**
        - **`asyncio.CancelledError`**
        - **`asyncio.TimeoutError`**：和`Exception.OSError.TimeoutError`区分开
        - `asyncio.InvalidStateError`：`Task/Future`内部状态无效引发
    - `asyncio.LimitOverrunError`：超出缓冲区引发的异常
    - `StopIteration`：`next()、send()`引发的异常：
        - `https://www.cnblogs.com/dotnetcrazy/p/9278573.html#6.Python迭代器`
    - `StopAsyncIteration`：`__anext__()`引发的异常
    - ArithmeticError
        - FloatingPointError
        - OverflowError
        - ZeroDivisionError
    - `AssertionError`：当断言`assert`语句失败时引发
    - `AttributeError`：当属性引用或赋值失败时引发
    - BufferError
    - `EOFError`
        - `asyncio.IncompleteReadError`：读取操作未完成引发的错误
    - ImportError
        - ModuleNotFoundError
    - LookupError
        - IndexError
        - KeyError
    - MemoryError
    - NameError
        - UnboundLocalError
    - **`OSError`**：当系统函数返回与系统相关的错误时引发
        - BlockingIOError
        - ChildProcessError
        - ConnectionError
           - BrokenPipeError
           - ConnectionAbortedError
           - ConnectionRefusedError
           - ConnectionResetError
        - FileExistsError
        - FileNotFoundError
        - InterruptedError
        - IsADirectoryError
        - NotADirectoryError
        - PermissionError
        - ProcessLookupError
        - **`TimeoutError`**：系统函数执行超时时触发
    - `ReferenceError`：引用错误（对象被资源回收或者删除了）
    - **`RuntimeError`**：出错了，但是检测不到错误类别时触发
        - `NotImplementedError`：为实现报错（比如调用了某个不存在的子类方法）
        - `RecursionError`：递归程度太深引发的异常
        - `asyncio.SendfileNotAvailableError`：系统调用不适用于给定的套接字或文件类型
    - **`SyntaxError`**：语法错误时引发（**粘贴代码经常遇到**）
        - `IndentationError`：缩进有问题
        - `TabError`：当缩进包含不一致的制表符和空格使用时引发
    - SystemError
    - `TypeError`：类型错误
    - ValueError
        - UnicodeError
        - UnicodeDecodeError
        - UnicodeEncodeError
        - UnicodeTranslateError
    - Warning
    - DeprecationWarning
    - PendingDeprecationWarning
    - RuntimeWarning
    - SyntaxWarning
    - UserWarning
    - FutureWarning
    - ImportWarning
    - UnicodeWarning
    - BytesWarning
    - ResourceWarning

### 3.5.6.任务分组、取消

### 3.5.7.兼容旧代码

### 3.5.8.同步语系列

### 3.5.9.socket新用法



### 关于源码的说明

之前并发编程的基础知识已经讲的很清楚了，也分析了很多源码，你可以自己去拓展一下（`Python3`的`asyncio`模块的源码一直在优化改进的路上）我这边就不一一分析了，你可以参考部分源码解析：<https://github.com/lotapp/cpython3/tree/master/Lib/asyncio>
![7.源码.png](https://img2018.cnblogs.com/blog/1127869/201901/1127869-20190107142010157-1086973041.png)

## 4.asynchttp

### 4.1.http扩展

### 4.2.爬虫小案例

asyncio是python用于解决异步IO编程的一整套解决方案

eg：

Django、Flask阻塞式IO，一般不会直接部署（它自带的解决方案只是方便调试），一般使用uwsgi，gunicorn + nginx来部署

tornado可以直接部署，